In [54]:
import datetime
from glob import glob
import os

import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

from rasterio.warp import (
    calculate_default_transform,
    reproject,
    Resampling
)

from netCDF4 import Dataset

from geocube.api.core import make_geocube

import xarray as xr
from rioxarray.rioxarray import _make_coords

import requests
from matplotlib import pyplot as plt
import h5py
from tqdm import tqdm

from multiprocessing import Pool

In [10]:
DATAPATH = "/media/user/dsk0/bggo/cmaqProjectdata"
URL = 'http://apis.data.go.kr/B552584/MsrstnInfoInqireSvc/getMsrstnList'
KEY = "k5wXUhoJHwee1cncQCBmm81YbQ+exttb0vdJcyF5GuGJn0mbGBNNL/ER2VfkrJMlExfc+FZjPeRuOM2bvgDYyQ=="

In [42]:
def get_gzpath_info() -> pd.DataFrame:
    emiss_s_list = glob(os.path.join(DATAPATH,"Emission","*Jul.tar.gz",))
    emiss_s_num = [os.path.split(path)[-1].split("_")[1] for path in emiss_s_list]

    emiss_info_df = pd.DataFrame()
    emiss_info_df.loc[:,'path'] = emiss_s_list
    emiss_info_df.loc[:,'s_num'] = emiss_s_num

    concentration_s_list = glob(os.path.join(DATAPATH,"Concentration","*.tar.gz",))
    concentration_s_num = [os.path.split(path)[-1].split("_")[1] for path in concentration_s_list]

    concentration_info_df = pd.DataFrame()
    concentration_info_df.loc[:,'path'] = concentration_s_list
    concentration_info_df.loc[:,'s_num'] = concentration_s_num

    all_conc_emis_pathinfo = pd.merge(emiss_info_df,concentration_info_df,how='left',on='s_num', suffixes=['_emission','_concentration'])
    all_conc_emis_pathinfo.s_num = all_conc_emis_pathinfo.s_num.astype(int)
    all_conc_emis_pathinfo = all_conc_emis_pathinfo.sort_values(by='s_num')
    all_conc_emis_pathinfo = all_conc_emis_pathinfo.reset_index(drop=True)

    return all_conc_emis_pathinfo

all_conc_emis_pathinfo = get_gzpath_info()
all_conc_emis_pathinfo

,path_emission,s_num,path_concentration
0,/media/user/dsk0/bggo/cmaqProjectdata/Emission...,1,/media/user/dsk0/bggo/cmaqProjectdata/Concentr...
1,/media/user/dsk0/bggo/cmaqProjectdata/Emission...,2,/media/user/dsk0/bggo/cmaqProjectdata/Concentr...
2,/media/user/dsk0/bggo/cmaqProjectdata/Emission...,3,/media/user/dsk0/bggo/cmaqProjectdata/Concentr...
3,/media/user/dsk0/bggo/cmaqProjectdata/Emission...,4,/media/user/dsk0/bggo/cmaqProjectdata/Concentr...
4,/media/user/dsk0/bggo/cmaqProjectdata/Emission...,5,/media/user/dsk0/bggo/cmaqProjectdata/Concentr...
...,...,...,...
113,/media/user/dsk0/bggo/cmaqProjectdata/Emission...,114,/media/user/dsk0/bggo/cmaqProjectdata/Concentr...
114,/media/user/dsk0/bggo/cmaqProjectdata/Emission...,115,/media/user/dsk0/bggo/cmaqProjectdata/Concentr...
115,/media/user/dsk0/bggo/cmaqProjectdata/Emission...,116,/media/user/dsk0/bggo/cmaqProjectdata/Concentr...
116,/media/user/dsk0/bggo/cmaqProjectdata/Emission...,118,/media/user/dsk0/bggo/cmaqProjectdata/Concentr...


In [43]:
def get_site_info() -> gpd.GeoDataFrame:
    params = {'serviceKey' : KEY, 'pageNo': 1, 'numOfRows': 640, 'returnType': 'json'}
    data = requests.get(URL, params=params, verify=False).json()
    site_info = pd.DataFrame(data['response']['body']['items'])

    def site_mapping(info):
        try: return Point(info[1].dmY, info[1].dmX)
        except: return None
    site_info['geometry'] = list(map(
        site_mapping,
        site_info.iterrows()
    ))
    site_info = site_info.loc[~site_info.isna().geometry.values]
    site_info = gpd.GeoDataFrame(site_info, geometry='geometry')
    site_info.crs = {'init':'epsg:4326'}
    return site_info

site_info = get_site_info()
site_info.head()

/home/yhlee/.local/share/virtualenvs/cmaq_datasets-Dn30YExT/lib/python3.9/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,dmX,item,mangName,year,addr,stationName,dmY,geometry
0,36.592906,"SO2, CO, O3, NO2, PM10, PM2.5",도시대기,2015,세종 조치원읍 군청로 87-16(신흥동) 세종특별자치시 조치원청사 옥상,신흥동,127.292253,POINT (127.29225 36.59291)
1,36.51212,"SO2, CO, O3, NO2, PM10, PM2.5",도시대기,2014,세종특별자치시 보듬3로 114 아름동커뮤니티센터 옥상 (아름동),아름동,127.24643,POINT (127.24643 36.51212)
2,36.474172,"SO2, CO, O3, NO2, PM10, PM2.5",도시대기,2018,"세종특별자치시 누리로 27 첫마을 6단지 관리사무소 옥상 (한솔동, 첫마을6단지)",한솔동,127.252529,POINT (127.25253 36.47417)
3,36.527029,"SO2, CO, O3, NO2, PM10, PM2.5",도시대기,2018,세종특별자치시 부강면 부강외천로 20 문화복지회관 옥상,부강면,127.370516,POINT (127.37052 36.52703)
4,36.479917,"SO2, CO, O3, NO2, PM10, PM2.5",도로변대기,None,세종특별자치시 한누리대로 2107 보람종합복지센터 지상 (보람동),보람동,127.292417,POINT (127.29242 36.47992)


In [44]:
resolution = 9000
x_dim, y_dim = 68, 83
projout = '+proj=lcc +lat_1=30 +lat_2=60 +lon_1=126 +lat_0=38 +lon_0=126 +ellps=GRS80 +units=m'
x = np.arange(-180_000, -180_000 + resolution * x_dim, resolution, dtype=np.float32)
y = np.arange(-585_000, -585_000 + resolution * y_dim, resolution, dtype=np.float32)
x_m, y_m = np.meshgrid(x, y)
G = np.dstack([x_m, y_m]).reshape(-1, 2)
grid_points = list(map(Point, G))

grid_data = pd.DataFrame(grid_points, columns=['geometry'])
grid_data = gpd.GeoDataFrame(grid_data, geometry='geometry')
grid_data.crs = site_info.to_crs(projout).crs
grid_data.loc[:,'x_m'] = grid_data.geometry.x
grid_data.loc[:,'y_m'] = grid_data.geometry.y
grid_data.loc[:,'value'] = 0

grid_data

,geometry,x_m,y_m,value
0,POINT (-180000.000 -585000.000),-180000.0,-585000.0,0
1,POINT (-171000.000 -585000.000),-171000.0,-585000.0,0
2,POINT (-162000.000 -585000.000),-162000.0,-585000.0,0
3,POINT (-153000.000 -585000.000),-153000.0,-585000.0,0
4,POINT (-144000.000 -585000.000),-144000.0,-585000.0,0
...,...,...,...,...
5639,POINT (387000.000 153000.000),387000.0,153000.0,0
5640,POINT (396000.000 153000.000),396000.0,153000.0,0
5641,POINT (405000.000 153000.000),405000.0,153000.0,0
5642,POINT (414000.000 153000.000),414000.0,153000.0,0


In [45]:
base_grid = make_geocube(vector_data=grid_data, measurements=["value"],resolution=(resolution, resolution), fill=0, output_crs=projout)
base_grid_rio = base_grid["value"].rio

base_crs = '+proj=lcc +lat_1=30 +lat_2=60 +lon_1=126 +lat_0=38 +lon_0=126 +ellps=GRS80 +units=m'

base_transform, base_width, base_height = calculate_default_transform(
    base_grid_rio.crs, base_crs, base_grid_rio.width, base_grid_rio.height, *base_grid_rio.bounds()
)
base_coords = _make_coords(
    base_grid, base_transform, base_width, base_height, base_crs
)
base_xs, base_ys = base_coords["x"], base_coords["y"]

base_grid

<xarray.Dataset>
Dimensions:      (y: 82, x: 67)
Coordinates:
  * y            (y) float64 -5.805e+05 -5.715e+05 ... 1.395e+05 1.485e+05
  * x            (x) float64 -1.755e+05 -1.665e+05 ... 4.095e+05 4.185e+05
    spatial_ref  int64 0
Data variables:
    value        (y, x) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0

In [49]:
def get_emission_list(s_num:str) -> pd.DataFrame:
        emission_info_df = pd.DataFrame()
        emission_list = glob(os.path.join(DATAPATH, "extract", "emission", f"RSM_{s_num}", "*"))
        emission_day_list = [os.path.split(path)[-1].split('.')[-5] for path in emission_list]
        emission_day_list_datetime = list(map(lambda date: datetime.date(int(date[:4]), int(date[4:6]), int(date[6:])), emission_day_list))
        emission_info_df.loc[:,['date']] = emission_day_list_datetime
        emission_info_df.loc[:,['path']] = emission_list
        return emission_info_df

def get_concentration_list(s_num:str) -> pd.DataFrame:
    concentration_info_df = pd.DataFrame()
    concentration_list = glob(os.path.join(DATAPATH, "extract", "concentration", f"RSM_{s_num}", "*"))
    concentration_day_list = [os.path.split(path)[-1].split('.')[-2] for path in concentration_list]
    concentration_day_list_datetime = []
    for date in concentration_day_list:
        start_date = datetime.date(int(date[:4]), 1, 1) + datetime.timedelta(-1)
        d_day = int(date[4:])
        target_date = start_date + datetime.timedelta(d_day)
        concentration_day_list_datetime.append(target_date)
    concentration_info_df.loc[:,['date']] = concentration_day_list_datetime
    concentration_info_df.loc[:,['path']] = concentration_list
    concentration_info_df = concentration_info_df.sort_values('date')
    concentration_info_df.index = range(len(concentration_info_df))
    return concentration_info_df

def murge_info(left:pd.DataFrame, right:pd.DataFrame) -> pd.DataFrame:
    conc_emis_info_df = pd.merge(
        left     = left, 
        right    = right,
        how      = "left", 
        on       = "date", 
        suffixes = ['_concentration', '_emission']
    )
    conc_emis_info_df_dropna = conc_emis_info_df.loc[~conc_emis_info_df.path_emission.isna(), ]
    conc_emis_info_df_dropna.index = range(len(conc_emis_info_df_dropna))
    return conc_emis_info_df_dropna

In [82]:
dataset_column = ['year', 'month', 'day', 'hour', 'weather_t', 'air_q_t', 'cmaq_t', 'smoke_t']
pm_10_cal = "(1.0*ASO4J[1])+(1.0*ASO4I[1])+(1.0*ANH4J[1])+(1.0*ANH4I[1])+(1.0*ANO3J[1])+(1.0*ANO3I[1])+(1.0*AALKJ[1])+(1.0*AXYL1J[1])+(1.0*AXYL2J[1])+(1.0*AXYL3J[1])+(1.0*ATOL1J[1])+(1.0*ATOL2J[1])+(1.0*ATOL3J[1])+(1.0*ABNZ1J[1])+(1.0*ABNZ2J[1])+(1.0*ABNZ3J[1])+(1.0*ATRP1J[1])+(1.0*ATRP2J[1])+(1.0*AISO1J[1])+(1.0*AISO2J[1])+(1.0*AISO3J[1])+(1.0*ASQTJ[1])+(1.0*AORGCJ[1])+(1.0*AORGPAJ[1])+(1.0*AORGPAI[1])+(1.0*AECJ[1])+(1.0*AECI[1])+(1.0*A25J[1])+(1.0*ANAJ[1])+(1.0*ACLJ[1])+(1.0*ACLI[1])+(1.0*AOLGAJ[1])+(1.0*AOLGBJ[1])+(1.0*ACORS[1])+(1.0*ASOIL[1])+(1.0*ANAK[1])+(1.0*ACLK[1])+(1.0*ASO4K[1])+(1.0*ANH4K[1])+(1.0*ANO3K[1])"
pm_10_chem_list = [chem[5:-4] for chem in pm_10_cal.split("+")]

import time

with h5py.File("hdf5/test.hdf5", "w") as f:
    for i in range(4): 
        s_num = all_conc_emis_pathinfo.s_num[i]
        concentration_info_df = get_concentration_list(s_num)
        emission_info_df = get_emission_list(s_num)
        merged_df = murge_info(concentration_info_df, emission_info_df)

        for i in range(len(merged_df)):
            nowdate = merged_df.loc[i].date
            #print(f"\r{i+1}/{len(merged_df)} {nowdate}", end="")
            conc_data = xr.open_dataset(merged_df.loc[i].path_concentration)
            emis_data = xr.open_dataset(merged_df.loc[i].path_emission)

            pm10_chem_map = map(lambda chem: conc_data[chem], pm_10_chem_list)
            smoke_chem_list = list(set(emis_data.variables.keys()) - set(['TFLAG']))
            print(time.time())
            smoke_chem_map = map(lambda chem: emis_data[chem], smoke_chem_list)
            print(time.time())

            conc_data.close()
            emis_data.close()

            day_concen_pm10 = np.sum(list(pm10_chem_map), axis=0, dtype=np.float64)

            day_concen_pm10 = day_concen_pm10.reshape(day_concen_pm10.shape[0], day_concen_pm10.shape[2], day_concen_pm10.shape[3], -1)

            day_concen_pm10 = day_concen_pm10.astype(np.float64)

            day_smoke_allval = np.concatenate(list(smoke_chem_map), axis=3, dtype=np.float64)[:day_concen_pm10.shape[0]]

            for hour_idx in range(24):
                group_key = f'{nowdate.year}/{nowdate.month}/{nowdate.day}/{hour_idx}'
                f[f'{group_key}/concentration'] = day_concen_pm10[hour_idx:hour_idx+1]
                f[f'{group_key}/smoke'] = day_smoke_allval[hour_idx:hour_idx+1]

        
        print()

conc_data

1677550907.7067711
1677550907.706827
1677550908.3113215
1677550908.3113894
1677550910.881034
1677550910.8811002
1677550913.3838825
1677550913.3839433
1677550915.187512
1677550915.1875756
1677550917.4564352
1677550917.4564953
1677550919.6916087
1677550919.6916695
1677550921.849113
1677550921.849184
1677550924.0612254
1677550924.06129
1677550926.3015132
1677550926.3015819
1677550928.705877
1677550928.70594
1677550931.088268
1677550931.0883322
1677550933.5515785
1677550933.5516393
1677550935.4983149
1677550935.49838
1677550937.6204453
1677550937.6205115
1677550940.0182064
1677550940.0182757
1677550942.272504
1677550942.272568
1677550944.3008657
1677550944.3009303
1677550946.4529576
1677550946.4530218
1677550953.1661282
1677550953.166192


KeyboardInterrupt: 

In [80]:
with h5py.File("hdf5/test.hdf5", "r") as f:
    print(f["2012/12/22/1/concentration"])
    print(f["2012/12/22/1/smoke"])

<HDF5 dataset "concentration": shape (1, 82, 67, 1), type "<f8">
<HDF5 dataset "smoke": shape (1, 19, 82, 3015), type "<f8">


In [66]:
conc_data["TFLAG"]

<xarray.DataArray 'TFLAG' (TSTEP: 24, VAR: 153, DATE-TIME: 2)>
[7344 values with dtype=int32]
Dimensions without coordinates: TSTEP, VAR, DATE-TIME
Attributes:
    units:      <YYYYDDD,HHMMSS>
    long_name:  TFLAG           
    var_desc:   Timestep-valid flags:  (1) YYYYDDD or (2) HHMMSS             ...